---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    ident = list()
    
    for G in P1_Graphs:
        degrees_graph = G.degree()
        degrees_graph_values = sorted(set(degrees_graph.values()))
        hist = [list(degrees_graph.values()).count(i)/float(nx.number_of_nodes(G)) for i in degrees_graph_values]
        
        avg_cl = nx.average_clustering(G)
        s_path = nx.average_shortest_path_length(G)
        
        if len(hist)>10:
            ident.append('PA')
            continue
        
        if s_path < 10 and avg_cl < 0.4:
            ident.append('SW_H')
        elif s_path < 10 and avg_cl > 0.4:
            ident.append('SW_L')
    
    return ident # Your Answer Here
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [6]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(nx.degree(G))
    
    X = df[(df['ManagementSalary'] == 0.0) | (df['ManagementSalary'] == 1.0)]
    y = X['ManagementSalary']
    del X['ManagementSalary']
    
    test_hold = df[(df['ManagementSalary'] != 0.0) & (df['ManagementSalary'] != 1.0)]
    del test_hold['ManagementSalary']
    
    #norm = MinMaxScaler()
    #X_norm = norm.fit_transform(X)
    #test_hold_norm = norm.transform(test_hold)
    
    #X_train, X_test, y_train, y_test = train_test_split(X_norm, y, random_state = 0)
    
    #clf = LogisticRegression()
    #grid_values = {'C':[0.01, 0.1, 1, 10, 100]}
    #grid_clf = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    #grid_clf.fit(X_train, y_train)
    #y_decision_fn_scores_auc = grid_clf.predict_proba(X_test)[:,-1]
    #print('Logistic Regression Test')
    #print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
    #print('Grid best score (AUC): ', grid_clf.best_score_)
    
    #clf = LogisticRegression(C=100).fit(X_norm, y)
    #pred = clf.predict_proba(test_hold_norm)[:,-1]
    #ans = pd.Series(pred, index = test_hold.index)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    #clf = RandomForestClassifier(random_state=0)
    #grid_values = {'n_estimators':[5, 10, 15, 20, 30, 50], 'max_features':[1, 2], 'max_depth': [3, 5, 8, 10]}
    #grid_clf = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    #grid_clf.fit(X_train, y_train)
    #y_decision_fn_scores_auc = grid_clf.predict_proba(X_test)[:,-1]
    #print('Random Forest Classifier Test')
    #print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
    #print('Grid best score (AUC): ', grid_clf.best_score_)
    
    #clf = GradientBoostingClassifier(random_state=0)
    #grid_values = {'n_estimators':[5, 10, 15, 20, 30, 50], 'max_features':[1, 2], 'max_depth': [3, 5, 8, 10], 'learning_rate':[0.01, 0.1, 0.5, 1, 5, 10]}
    #grid_clf = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    #grid_clf.fit(X_train, y_train)
    #y_decision_fn_scores_auc = grid_clf.predict_proba(X_test)[:,-1]
    #print('Gradient Boosting Classifier Test')
    #print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
    #print('Grid best score (AUC): ', grid_clf.best_score_)
    
    clf = GradientBoostingClassifier(random_state=0, learning_rate=0.1, max_depth=3, n_estimators=15, max_features=2)
    clf.fit(X, y)
    pred = clf.predict_proba(test_hold)[:, -1]
    prob_pred = pd.Series(pred, index=test_hold.index)
    
    return prob_pred # Your Answer Here
salary_predictions()

1       0.108453
2       0.551687
5       0.830030
8       0.220713
14      0.153176
18      0.117773
27      0.114727
30      0.439705
31      0.402976
34      0.126439
37      0.220713
40      0.114727
45      0.105619
54      0.489880
55      0.287220
60      0.439705
62      0.830030
65      0.492860
77      0.069587
79      0.108453
97      0.074819
101     0.067834
103     0.402976
108     0.116241
113     0.408962
122     0.063058
141     0.178938
142     0.719272
144     0.114528
145     0.483518
          ...   
913     0.074819
914     0.074819
915     0.049516
918     0.126439
923     0.090195
926     0.090195
931     0.074819
934     0.087806
939     0.056768
944     0.044571
945     0.090195
947     0.090195
950     0.135625
951     0.074819
953     0.056768
959     0.049516
962     0.049516
963     0.205843
968     0.090195
969     0.090195
974     0.090195
984     0.087806
987     0.131842
989     0.090195
991     0.136858
992     0.049516
994     0.044571
996     0.0495

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
df = future_connections.copy()
df['PA'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
df['CN'] = df.index.map(lambda number: len(list(nx.common_neighbors(G, number[0], number[1]))))
df['JC'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
#df['RA'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
df.head()

,Future Connection,PA,CN,JC
"(6, 840)",0.0,2070,9,0.073770
"(4, 197)",0.0,3552,2,0.015504
"(620, 979)",0.0,28,0,0.000000
"(519, 872)",0.0,299,2,0.060606
"(382, 423)",0.0,205,0,0.000000


In [9]:
def new_connections_predictions():
    
    # Your Code Here
    X = df[(df['Future Connection']==0.0) | (df['Future Connection']==1.0)]
    y = X['Future Connection']
    test_hold = df[(df['Future Connection']!=0.0) & (df['Future Connection']!=1.0)]
    del test_hold['Future Connection']
    del X['Future Connection']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    #clf = RandomForestClassifier(random_state=0)
    #grid_values = {'n_estimators':[10, 15, 20, 30], 'max_features':[1, 2]}
    #grid_clf = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    #grid_clf.fit(X_train, y_train)
    #y_proba_fn_scores_auc = grid_clf.predict_proba(X_test)[:,-1]
    #print('Random Forest Classifier Test')
    #print('Test set AUC: ', roc_auc_score(y_test, y_proba_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
    #print('Grid best score (AUC): ', grid_clf.best_score_)
    
    #clf = GradientBoostingClassifier(random_state=0)
    #grid_values = {'n_estimators':[10, 15, 20, 30], 'max_features':[1, 2], 'learning_rate':[0.01, 0.1, 0.5, 1]}
    #grid_clf = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    #grid_clf.fit(X_train, y_train)
    #y_proba_fn_scores_auc = grid_clf.predict_proba(X_test)[:,-1]
    #print('Gradient Boosting Classifier Test')
    #print('Test set AUC: ', roc_auc_score(y_test, y_proba_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
    #print('Grid best score (AUC): ', grid_clf.best_score_)
    
    clf = GradientBoostingClassifier(random_state=0, learning_rate=0.1, n_estimators=30, max_features=2)
    clf.fit(X, y)
    pred = clf.predict_proba(test_hold)[:, -1]
    prob_pred = pd.Series(pred, index=test_hold.index)
    
    return prob_pred # Your Answer Here
new_connections_predictions()

(107, 348)    0.028590
(542, 751)    0.017328
(20, 426)     0.559791
(50, 989)     0.017328
(942, 986)    0.017328
(324, 857)    0.017328
(13, 710)     0.103269
(19, 271)     0.130340
(319, 878)    0.017328
(659, 707)    0.017328
(49, 843)     0.017328
(208, 893)    0.017328
(377, 469)    0.017328
(405, 999)    0.020307
(129, 740)    0.017858
(292, 618)    0.020307
(239, 689)    0.017328
(359, 373)    0.017328
(53, 523)     0.031569
(276, 984)    0.017328
(202, 997)    0.017328
(604, 619)    0.138946
(270, 911)    0.017328
(261, 481)    0.075710
(200, 450)    0.817734
(213, 634)    0.017328
(644, 735)    0.104610
(346, 553)    0.017328
(521, 738)    0.017328
(422, 953)    0.020307
                ...   
(672, 848)    0.017328
(28, 127)     0.939034
(202, 661)    0.017328
(54, 195)     0.949018
(295, 864)    0.017328
(814, 936)    0.017328
(839, 874)    0.017328
(139, 843)    0.017328
(461, 544)    0.017328
(68, 487)     0.017328
(622, 932)    0.017328
(504, 936)    0.020307
(479, 528) 